## Setup
First of all, user needs open a notebook page by "Small Rectangle", Jupyter or JupyterLab

### 1. Install the package

In [ ]:
!pip install azureml-sparkmagic

### 2. (Optional) Enable progress indicator

In [ ]:
# For JupyterLab, it needs another command
!jupyter nbextension enable --user --py azureml.sparkmagic.progress_indicator

Enabling notebook extension sparkmagic/spark-progress-indicator/module...
      - Validating: ok


### 3. (Optional) Enable auto-load

In [ ]:
# This command will override existing ipython config
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('azureml.sparkmagic.magics')" >>  $(ipython profile locate default)/ipython_kernel_config.py

## Start to work
### 1. Load sparkmagic if auto-load is not enbled

In [ ]:
%load_ext azureml.sparkmagic.magics

### 2. Start Spark Session
Start a new session will stop current running session if any.

In [ ]:
# You can change to any Synapse workspace you can access
%spark start --synapse hochsynapse --sparkpool hochpool

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code XXXXXXXXX to authenticate.
Starting Spark application ...
SparkSession available as 'spark'.
SparkContext available as 'sc'.


### 3. Submit Code

In [ ]:
%%spark

# get workspace
from azureml.core import Workspace
subscription_id = '35f16a99-532a-4a47-9e93-00305f6c40f2'
resource_group = 'arthfa_test'
workspace_name = 'arthfa_test_azureml_ws'
workspace = Workspace(subscription_id, resource_group, workspace_name)

# get dataset
from azureml.core import Dataset
ds = Dataset.get(workspace, name = 'movies-yang')
df = ds.to_pandas_dataframe() 

# apply spark transforms
from  pyspark.sql.functions import desc
spark_df = sqlContext.createDataFrame(df)
spark_df.groupBy('Distributor').count().orderBy(desc('count')).show(10)

+--------------------+-----+
|         Distributor|count|
+--------------------+-----+
|        Warner Bros.|  318|
|       Sony Pictures|  307|
|  Paramount Pictures|  257|
|           Universal|  254|
|Walt Disney Pictures|  232|
|                    |  232|
|    20th Century Fox|  229|
|                 MGM|  173|
|             Miramax|  137|
|            New Line|  136|
+--------------------+-----+
only showing top 10 rows


### 4. Show Session Status
When user want to see spark page, this command can provide the url.

In [ ]:
%spark status

{'SessionId': '181', 'ApplicationId': 'application_1584322909487_0001', 'State': 'idle', 'Url': 'https://web.azuresynapse.net/monitoring/sparkapplication/aml34214?workspace=/subscriptions/35f16a99-532a-4a47-9e93-00305f6c40f2/resourceGroups/hoch/providers/Microsoft.Synapse/workspaces/hochsynapse&livyId=181&sparkPoolName=hochpool'}


### 5. Stop Session
Usually, user don't need to do this explicitly. The session can be shutdown automatically after timeout.

In [ ]:
%spark stop

### 6. Set and Get Session Config
The new config will apply to new session. 

In [ ]:
%%spark config
{
    "driverMemory":"8g",
    "driverCores":2,
    "executorMemory":"8g",
    "executorCores":2,
    "numExecutors":2
}

{'driverMemory': '8g', 'driverCores': 2, 'executorMemory': '8g', 'executorCores': 2, 'numExecutors': 2}
